# Pseudo-Synthetic Benchmarking

## (Or) Finding holes in benchmarks: Evaluating the coverage and corresponding performance predictions for conventional vs synthetic benchmarking

### Introduction

Accelerators – or dedicated hardware which accelerates specific types of computation – are of increasing importance for both performance and energy-efficient computing.
For instance, as of June 2018 [@dongarra_2018], 7 of the top 10 of supercomputers from the Top500 are composed of at least one accelerator per node.
There are 10-100k nodes in any of these super computers and the energy usage of these machines scale accordingly – from 1,600kW to 18,000kWs.
Given the adoption of accelerators in these machines and the corresponding energy benefits of using the most appropriate one for a given code, it is likely that a wider diversity of accelerators per node will occur.
Indeed, the latest IBM Power 9 CPU architecture features both NVLINK and CAPI interconnects, these facilitate high-speed intra-node level memory transfers between NVIDIA devices and devices from other vendors respectively.
Since this processor is present in both number 1 – Summit – and 3 – Sierra – of the Top500, it is forseeable that the addition of FPGAs and AMD GPUs is likely, especially as these devices advance.

We motivate our work with this platform in mind, namely, we evaluate the performance of a model capable of highly accurate run-time predictions of codes without them having been run
on any specific device.
The metrics used to perform prediction are device agnostic and are generated once, post-development, and are included as a comment in the code to be used during scheduling.
The model was developed by measuring the execution times of OpenCL kernels – from the Extended OpenDwarfs Benchmark Suite – on a wide range of devices.
The metrics were treated as independent or input variables whist the execution times were treated as dependent or response variables in a random forest regression model.

An evaluation of this model was performed by considering a set of synthetic benchmarks randomly generated using a Machine-Learning OpenCL kernel generation framework – CLgen.
These extensive set of kernel benchmarks were used to automatically produce a wide range of codes based on the finite/limited OpenCL applications available on github.

The previous success of this model to predict execution times simultaneously across many devices with high accuracy has lead us to believe that the Extended OpenDwarfs Benchmark
Suite is a good platform for training – it adequately covers the feature space for scientific problems typical of the HPC setting.
However, testing the model with synthetic benchmarking identifies gaps in the selected applications, since they are poorly predicted in the model.

This work uses CLgen -- an open source application for generating runnable programs using deep learning which learns to program using neural networks which model the semantics and usage from large volumes of program fragments, generating many-core OpenCL programs that are representative of, but distinct from, the programs it learns from.
We use CLgen to synthesize kernels which are then incorporated into our predictive framework -- where the synthsized kernel is both predicted with a random-forest model on its AIWC feature-space -- and compared against directly executing the same synthetic kernel on a CPU and CPU accelerator.
If the prediction deviates significantly from the measured execution time we have found a "hole" in the benchmark suite, where conventional benchmarking (in this instance the EOD benchmark suite) has missed a common characteristic of applications in the wild.
This synthetic benchmark can then be added back into an augmented benchmark suite -- which contains a hybrid of conventional and synthetic benchmarks.
The augmented or pseudo-synthetic benchmark suite methodology will benefit the wider HPC community by both offering a comprehensive benchmark suite which is empericially shown to encapsulate all major patterns of computation and communication executed on accelerators as well as increasing the predictive performance of our previously release random forest model -- which will benefit the future of HPC scheduling.

### Results

This Jupyter workbook is our attempt to increase interpretability and reproducibility in our analysis and workflow to the wider high-performance computing community.
To synthesize a fresh batch of kernels run the following command:

In [1]:
! cd /phd; bazel run //deeplearning/clgen -- --config /workspace/codes/corpus/tiny_synthesizer_config.pbtxt

/bin/sh: 1: cd: can't cd to /phd
/bin/sh: 1: bazel: not found


Determine where the files have been generated -- thankfully this can be stored at the end of execution automatically with environment variables.

In [20]:
! cd /phd; bazel run //deeplearning/clgen -- --config /workspace/codes/synthetic-benchmark-driver/tiny_synthesizer_config.pbtxt --print_cache_path=sampler > .synthetic_kernel_directory
! export SYNTHESIZED_KERNEL_DIRECTORY=$(cat /phd/.synthetic_kernel_directory)
import os
with open('/phd/.synthetic_kernel_directory', 'r') as content_file:
    os.environ['SYNTHESIZED_KERNEL_DIRECTORY'] = content_file.read()

Loading: 
Loading: 0 packages loaded
INFO: Analysed target //deeplearning/clgen:clgen (0 packages loaded).
INFO: Found 1 target...
[0 / 3] [-----] BazelWorkspaceStatusAction stable-status.txt
Target //deeplearning/clgen:clgen up-to-date:
  bazel-phd/bazel-out/k8-py3-opt/bin/deeplearning/clgen/clgen
INFO: Elapsed time: 0.472s, Critical Path: 0.02s
INFO: 0 processes.
INFO: Build completed successfully, 1 total action
INFO: Running command line: bazel-phd/bazel-out/k8-py3-opt/bin/deeplearning/clgen/clgen --config /workspace/codes/synthetic-benchmark-driver/tiny_synthesizer_config.pbtxt '--print_cache_path=sampler'
INFO: Build completed successfully, 1 total action


In [22]:
!echo $SYNTHESIZED_KERNEL_DIRECTORY

/tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f


Ensure the driver program is up-to-date and compiled. It's responsible for generating different sized payloads, compiling each synthetic kernel and running them.

In [23]:
! cd /workspace/codes/synthetic-benchmark-driver; make

make: Nothing to be done for 'all'.


We can now run all the synthesized kernels in this temporary directory.

```
Arguments to the `sbd_over_directory.sh` script is:
    `<DIRECTORY>` the filepath to directory of kernels, 
    `<SIZE>` is the payload size (can be `tiny`, `small`, `medium` or `large`), 
    `<PLATFORM>` is the platform id (an integer specific to your platform -- run `!/opencl_device_query/opencl_device_query` for options),
    `<DEVICE>` is the device id (an integer specific to your platform -- again run `!/opencl_device_query/opencl_device_query` for options),
    `<MODE>` denotes the running mode (can be `aiwc` or `runtime`) `aiwc` means the synthetic kernel will be run once -- since AIWC is deterministic, while `runtime` will executed under a 100 repetitions -- for statistical runtime sampling.
```


In [24]:
! cd /workspace/codes/synthetic-benchmark-driver; ./sbd_over_directory.sh $SYNTHESIZED_KERNEL_DIRECTORY tiny 1 0 runtime

Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/0387ea63a01ed6fccc7e05da8e0ceef1bcf0226f1a8ce5036387188f05a17ae0.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = i + 0;\n  int h = f;\n  int f = get_local_id(0);\n  int j = get_global_id(1);\n  int l = get_group_id(0);\n  int j = get_local_id(1);\n\n  if (j = 0x1f;\n\n  float m = 0.f;\n  int o;\n  for (k = 0; k < e; k++) {\n    for (uint k = e[j + 1]; k += h; k = k; o++o) {\n    int q = f / 16;\n\n    float m = d.x + j * c;\n    float m = 0.0f;\n\n    float m = c.m_se i[k * 2 - g + 1;\n    float q = d[o + r * b + g];\n    float q = 0.f;\n\n    int o = e[p];\n\n    f += d[g] - c[p + s * o];\n    inv si;\n    ao = s[aq + ao];\n\n    int aa = k[au % 4 + ae * 4];\n\n    for (int m = 0; m < e; m++) {\n      n = p;\n    }\n\n  for (uint k = 0;\n    for (uint o = 0; m < i - l * h; ++j) {\n      f

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 13218 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/092b3e4363be3179a985d48840917fb778c7b891b70b5489eb975977edad7f2c.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = get_global_id(0);\n  int e = get_group_id(0);\n  int f = get_local_id(1);\n\n  int h = get_local_id(1);\n\n  if (p < h) {\n    int r = j[l];\n\n    for (int q = 0; m < i + 16; k++) {\n      for (uint k = e; m < d[j]; ++l)\n        q += (((*d) * size(n + i + o + l + o) > n) {\n          i = (*d);\n        i = k * e + f[q] + c[l * e + e];\n    }\n  }\n\n  g[0].s1 = o;\n  n = j;\n  w = x.s0 * q.s0 + p.s1 + q.s1 + q.s5 + o.s2 + o.s6 + q.s7 + o.s7 + q.s7 + q.x + (a

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 13350 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/119b625cc8d8251ecee8eb6ba91316a1b16a91346a9a4708730c7fb9c652af62.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = get_local_id(0) + (32)) == 1);\n  if ((j > 0) {\n    const rest = (mloat16)(- 0x1.9910163}ps34 + D - l, q, q, q, q, 0, 0);\n\n    if (((p + m.m + k / (f[k + k + j], & 0); p > d + (j[j] + i[(0 + n + j + m] > c) % (32 + k - 1.0 + 1) * c + k * 1] = m[(> > 1.0);\n\n    p = a[o + q * o] * a[p + r * o];\n    j = p * s + q - q * 3;\n\n    ab[0] = a[q * 3 + 1] * a[0 + a[1 + e * 16] * n + c[o + 9] + p + l[q + s * o];\n    a[n] = a[n] + b[r];\n    a[l] += h[l * h];\n   

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 13515 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/1538e939a1401f7767ecce093eeebd1dd18469e41eb8906e4b0cc2f46dfa8b64.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = A(d, -1);\n\n  float e = a[3] + a[0] + a[4];\n\n  float g = 1.h >> 1;\n\n  for (uint s = (d + 1));\n  size_t i = b - 1;\n\n  int j = i % (i / l));\n  local float* l = (float)(f) / 4);\n  const uint j = get_global_id(0);\n  if (e < e && l < h) {\n    g = (float)(p)\n      i + n = (f[m], d[p + s * d]);\n    c[g] += c[g + 1] + c[l + 1] + c[l + 13] + c[p + 13] + c[o + 13] + c[o + 13] + c[o + 13] + c[l - 13] + c[l + 14] + e[l + e], e[2 * m + m + 16] + c[o + 15];\n\

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 13680 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/1a0383b7b2e4f91211370232584ecbbfd730767076bd17cf7712aa8e609cac4b.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = A(d, -1);\n\n  double (d = c / 1) - d > 1 ? 1 : 0 == 16 {\n    return;\n  }\n\n  e[g * c + j] = (d.0 / 2, c, c, d, e);\n\n  barrier(1);\n\n  if (e == 256 - 1) {\n    for (int s = 0; m < 160; k++) {\n      n = k + i;\n      t + r;\n    }\n    barrier(1);\n  }\n\n  if (g >= 24) {\n    c[e] += c[e + 1];\n    case 2: {\n      *b = 1;\n      *b = 0;\n      e.f = !a += e;\n    }\n    barrier(1);\n  }\n\n  if (g == 0 && c =< 2) {\n    s = 0;\n  }\n\n  g = j;\n  k = -

...

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 16914 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/ee033198d5f7e7ca1c0a4d5d88d069b90311d6aa70a5dee9413db7ed555912a2.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  size_t d = get_global_id(0);\n  c[e] = b[e] + c * c[e];\n\n  i = e.f;\n\n  if (c == 2 && d < c)\n    d = B(g, e, 10, 0);\n    float p = d[p];\n\n    for (int m = 0; m < d; m++)\n      f[p + q * o] = m[aq + o + r * o] = ((1.0 / 2.0) * c[q + r * o] + a[p + r + o] + 1, 6, d, c);\n    }
Attempting kernel: /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/ee033198d5f7e7ca1c0a4d5d88d069b90311d6aa

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 17013 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/f20f1b3bbed3dfc88a4891b37d676a018d9d43b4e43a3e3dcf2a750d390c3682.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = get_group_id(0) + 0024 / e]; d < c;\n  d += get_group_id(0) * 512U + get_local_id(0);\n  e[get_local_id(0) + 0] = c[0];\n  e[get_local_id(0) + 0] = c[4];\n}
Attempting kernel: /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/f20f1b3bbed3dfc88a4891b37d676a018d9d43b4e43a3e3dcf2a750d390c3682.pbtxt.cl with contents:
__kernel void A(__global float* a, __global float* b, __global float* 

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 17079 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/f34dce901050f9526993d20d5ab15a94549169528d27e779e091cf276a17213f.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = A(d, -1);\n  while (e >= 0) {\n    a[e] = b[e] / c;\n    e = A(d, d);\n    e[f] = k;\n  }\n}
Attempting kernel: /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/f34dce901050f9526993d20d5ab15a94549169528d27e779e091cf276a17213f.pbtxt.cl with contents:
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  int e = A(d, -1);\n  while (e >= 0) {\n    

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 17178 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/f7f8662349dce70ff2058c0e3b3bc307596ff879b0edfd9aee3e650776defefa.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\n  size_t c = get_global_id(0);\n  const int g = e * c;\n  int h = b / a;\n\n  int h = b;\n\n  if (e < c) {\n    float j = (((m > 0.0))\n      c[g] = e[e * c + j];\n\n    barrier(1);\n  }\n  if (get_global_id(1);\n\n  for (uint o = 0; o < 12; k+m) {\n    int o = e[p + p] + j;\n    float s = (x * sizeof(flof(00000000v_size(0))\n      int m = 156;\n    if (j < 16) {\n      c[e] += c[e + 1];\n    }\n    barrier(1);\n  }\n  if (g >= 64) {\n    if (e < 256) {\n      c[e] += c

1
2
terminate called after throwing an instance of 'std::runtime_error'
  what():  can't build program
./sbd_over_directory.sh: line 13: 17277 Aborted                 (core dumped) ./sbd $f.cl $SIZE $PLATFORM $DEVICE $MODE
Processing /tmp/phd/deeplearning/clgen/tiny/model/0460e71bb961523d3f8340685dd8b822551dbd35/samples/88bf044e7d87cbcb3e8e60cfc0d4ecfbc6f1f27f/fc0528807c632e3f3cf0e9ca0ca0b418a20ef49861f26c6d30e6cfe7dea27e86.pbtxt...
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {\ncol I(double* a, global double* c, const int c, unsigned int d, global const b, global float* d, global float* e, global float* f, global float* e, int f, global float* g, global float* h, global int4* u, int k, global const int i, local float* e, global float* i, int k, global int* s, int f, global int* s, global int* s) {\n  int i = get_global_id(0);\n  int l = get_global_id(0);\n  int j = get_local_id(1);\n\n  float j = (global float)* k * (get_local_id(0) - (float)m

# Paper Results

In August 2018 we scraped all of [Github](https://github.com/) for any OpenCL kernels -- the details of running the scraper tool is outlined in the associated [creating-a-corpus](.) notebook.
This has been provided as a tarball [](./corpus/full-github.tar.bz2) and contains ~3.2k kernel files.

In [21]:
!ls

AIWCFeatures			  generatedkernelfeatures
Dockerfile			  generatedkernels
KernelRuntimesP100		  kernels.db
KernelRuntimesXeonGold		  make-kernels-and-features.py
Synthesize-and-run-kernels.ipynb  paper-synthesizing-benchmarks-model
clgen-create-db			  predictions
clgen-fetch-github		  scrape-corpus
corpus				  synthesize_kernels.py
creating_a_corpus.ipynb		  synthetic-benchmark-driver
generatedkernelbatch1		  unzipall


In [40]:
import pandas as pd
import numpy as np
import re
import os
import copy
import clgen.clutil
import clgen.model
import sqlite3
import clgen.sampler
import clgen.dbutil
import clgen.explore
import re
import shutil
import os
from subprocess import Popen, PIPE
import glob


In [2]:
# Renames our dataframe from the default output format into format suitable for opencl predictions with AIWC
def renamecolumns(df):
    ndf = df[["application", "size", "opcode", "granularity", "barriers per instruction", "instructions per operand", \
       "total instruction count", "workitems", "operand sum", "total # of barriers hit", "min instructions to barrier", \
       "max instructions to barrier", "median instructions to barrier", "max simd width", "mean simd width", \
       "stdev simd width", "total memory footprint", "90% memory footprint", "global memory address entropy", \
       "local memory address entropy -- 1 LSBs skipped", "local memory address entropy -- 2 LSBs skipped", \
       "local memory address entropy -- 3 LSBs skipped", "local memory address entropy -- 4 LSBs skipped", \
       "local memory address entropy -- 5 LSBs skipped", "local memory address entropy -- 6 LSBs skipped", \
       "local memory address entropy -- 7 LSBs skipped", "local memory address entropy -- 8 LSBs skipped", \
       "local memory address entropy -- 9 LSBs skipped", "local memory address entropy -- 10 LSBs skipped", \
       "total unique branch instructions", "90% branch instructions", "branch entropy (yokota)", "branch entropy (average linear)", \
       "device", "total_time", "kernel_time"]]
    ndf.loc[:, "invocation"] = 0
    try:
        if (df['kernel'] == "P100"):
            ndf.loc[:, 'kernel'] = df['kernel']
        else:
            ndf.loc[:, "kernel"] = "CLGen Kernel"
    except:
        ndf.loc[:, "kernel"] = "CLGen Kernel"
    ndf.loc[:, 'kernel'] = df['kernel']
    ndf.loc[:, "run"] = np.arange(1,ndf.values.shape[0]+1, 1)
    ndf.columns = ["application", "size", "opcode", "granularity", "barriers_per_instruction", "instructions_per_operand", \
           "total_instruction_count", "workitems", "operand_sum", "total_barriers_hit", "min_instructions_to_barrier", \
           "max_instructions_to_barrier", "median_instructions_to_barrier", "max_simd_width", "mean_simd_width", \
           "stddev_simd_width", "total_memory_footprint", "ninety_percent_memory_footprint", "global_memory_address_entropy", \
           "local_memory_address_entropy_1", "local_memory_address_entropy_2", \
           "local_memory_address_entropy_3", "local_memory_address_entropy_4", \
           "local_memory_address_entropy_5", "local_memory_address_entropy_6", \
           "local_memory_address_entropy_7", "local_memory_address_entropy_8", \
           "local_memory_address_entropy_9", "local_memory_address_entropy_10", \
           "total_unique_branch_instructions", "ninety_percent_branch_instructions", "branch_entropy_yokota", "branch_entropy_average_linear", \
           "device", "total_time", "kernel_time", "invocation", "kernel", "run"]
    return ndf



In [3]:
renameDictionary = {
    "application":"application",
    "size":"size",
    "opcode":"opcode",
    "granularity":"granularity",
    "barriers per instruction":"barriers_per_instruction",
    "instructions per operand":"instructions_per_operand",
    "total instruction count":"total_instruction_count",
    "workitems":"workitems",
    "operand sum":"operand_sum",
    "total # of barriers hit":"total_barriers_hit",
    "min instructions to barrier":"min_instructions_to_barrier",
    "max instructions to barrier":"max_instructions_to_barrier",
    "median instructions to barrier":"median_instructions_to_barrier",
    "max simd width":"max_simd_width",
    "mean simd width":"mean_simd_width",
    "stdev simd width":"stddev_simd_width",
    "total memory footprint":"total_memory_footprint",
    "90% memory footprint":"ninety_percent_memory_footprint",
    "global memory address entropy":"global_memory_address_entropy",
    "local memory address entropy -- 1 LSBs skipped":"local_memory_address_entropy_1",
    "local memory address entropy -- 2 LSBs skipped":"local_memory_address_entropy_2",
    "local memory address entropy -- 3 LSBs skipped":"local_memory_address_entropy_3",
    "local memory address entropy -- 4 LSBs skipped":"local_memory_address_entropy_4",
    "local memory address entropy -- 5 LSBs skipped":"local_memory_address_entropy_5",
    "local memory address entropy -- 6 LSBs skipped":"local_memory_address_entropy_6",
    "local memory address entropy -- 7 LSBs skipped":"local_memory_address_entropy_7",
    "local memory address entropy -- 8 LSBs skipped":"local_memory_address_entropy_8",
    "local memory address entropy -- 9 LSBs skipped":"local_memory_address_entropy_9",
    "local memory address entropy -- 10 LSBs skipped":"local_memory_address_entropy_10",
    "total unique branch instructions":"total_unique_branch_instructions",
    "90% branch instructions":"ninety_percent_branch_instructions",
    "branch entropy (yokota)":"branch_entropy_yokota",
    "branch entropy (average linear)":"branch_entropy_average_linear",
    "device":"device",
    "total_time":"total_time",
    "kernel_time":"kernel_time"
}

def renamecolumns(df):
    for k in renameDictionary.keys():
        df = df.rename(columns={k:renameDictionary[k]})
    return df
        
# Reorders a dataframe so that the columns match up for predictions with AIWC
def orderNames(df):
    df['run'] = 0
    d = df[["application", "size", "opcode", "granularity", "barriers_per_instruction", "instructions_per_operand", \
           "total_instruction_count", "workitems", "operand_sum", "total_barriers_hit", "min_instructions_to_barrier", \
           "max_instructions_to_barrier", "median_instructions_to_barrier", "max_simd_width", "mean_simd_width", \
           "stddev_simd_width", "total_memory_footprint", "ninety_percent_memory_footprint", "global_memory_address_entropy", \
           "local_memory_address_entropy_1", "local_memory_address_entropy_2", \
           "local_memory_address_entropy_3", "local_memory_address_entropy_4", \
           "local_memory_address_entropy_5", "local_memory_address_entropy_6", \
           "local_memory_address_entropy_7", "local_memory_address_entropy_8", \
           "local_memory_address_entropy_9", "local_memory_address_entropy_10", \
           "total_unique_branch_instructions", "ninety_percent_branch_instructions", "branch_entropy_yokota", "branch_entropy_average_linear", \
           "device", "total_time", "kernel_time", "invocation", "kernel", "run"]]
    return d

In [44]:
def getfeatures():
    # reads in the aiwc fabseature space file from the A_0 csv file
    currentAIWC = "aiwc_A_0.csv"
    aiwcfeatures = pd.read_csv(currentAIWC).values.T
    aiwcfeaturesdf = pd.DataFrame(columns=list(aiwcfeatures[0,:]))
    aiwcfeaturesdf.loc[0,] = aiwcfeatures[1,:]
    aiwcfeaturesdf['tmp'] = 1
    return aiwcfeaturesdf

def getruntimes(psize, device):
    featuras = []
    counter = 0
    for f in glob.glob("lsb.*"):

        kernelfeatures = pd.read_csv(f, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
        headerinfo = pd.read_csv(f)

        
        kernelfeatures['device'] = device
        kernelfeatures['size'] = psize
        kernelfeatures['sample_number'] = counter

        # Remove non-kernels from our regions
        kernelfeatures = kernelfeatures[kernelfeatures['region'].str.contains("kernel")==True]
        kernelfeatures = kernelfeatures[kernelfeatures['region'].str.contains("kernel_creation")==False]
        
        # computes the mean times of those by id
        total_time = kernelfeatures.groupby(["id"]).mean().reset_index()
        kernelfeatures = kernelfeatures.drop(["repeats_to_two_seconds", 'time', 'sample_number', 'overhead'], axis=1)
        total_time = total_time.rename(columns = {"time" : "kernel_time"})
        kernelfeatures = pd.merge(kernelfeatures, total_time, how='inner', on='id')

        kernelfeatures['total_time'] = copy.deepcopy(kernelfeatures['kernel_time'])
        kernelfeatures = kernelfeatures.drop_duplicates('kernel_time')

        kernelfeatures['application'] = "CLGen Model"
        kernelfeatures['tmp'] = 1
        counter+=1
        featuras.append(kernelfeatures)
        
    allruntimes = pd.concat(featuras, axis=0)
    allruntimes['kernel_time'] = allruntimes.mean().values[2]
    allruntimes['total_time'] = copy.deepcopy(allruntimes['kernel_time'])
    allruntimes = allruntimes.drop_duplicates('kernel_time')
    
    return allruntimes

def concatandrename(runtimesdf, featuresdf):
    df = pd.merge(runtimesdf, featuresdf, on='tmp')
    df = renamecolumns(df)
    return df

In [ ]:
#!/usr/bin/env python3

version = clgen.version()
print("Successfully loaded CLgen version {}".format(version))

###################################
# Remove previously generated kernels
###################################
shutil.rmtree("generatedkernels/", ignore_errors=True)
shutil.rmtree("generatedkernelfeatures//", ignore_errors=True)

###################################
# Uses Chris Cumins model to generate kernels based on on batch size and max kernels
# Writes these kernels to a database file
###################################
clgen.clutil.platform_info()
model = clgen.model.from_tar("./paper-synthesizing-benchmarks-model/model.tar.bz2")
argspec = ['__global float*', '__global float*', '__global float*', 'const int']
sampler = clgen.sampler.from_json({
        "kernels": { 
            "args": argspec,
            "max_length": 1000
        },
        "sampler": {
            "batch_size": 100,
            "max_kernels": 50
        }
    })
print("Seed text:", clgen.sampler.serialize_argspec(argspec), "\n")
sampler.cache(model).empty()
sampler.sample(model)
db = sampler.cache(model)["kernels.db"]
num_good_kernels = clgen.dbutil.num_good_kernels(db)
clgen.explore.explore(db)

###################################
# Establish connection to database file and write the .cl files which pass the rejection filter
# to a folder called generatedkernels
###################################
conn = sqlite3.connect(db)
print(conn)
c = conn.cursor()
c.execute("SELECT * FROM PreprocessedFiles WHERE status=0;")
rows = c.fetchall()
if not(os.path.isdir("generatedkernels")):
    os.mkdir("./generatedkernels")

for row in rows:
    name = "./generatedkernels/" + str(row[0]) + ".cl"
    newf = open(name, "w+")
    newf.writelines(row[2])
    newf.close()

usable_kernels = []
unusable_kernels = []

if not(os.path.isdir("generatedkernelfeatures")):
    os.mkdir("./generatedkernelfeatures")
os.chdir("generatedkernelfeatures")

###################################
# Runs the kernels through the synthetic benchmark driver and checker
# Extracts AIWC features and runtimes to generatekernelfeatures folder
###################################
 
# Synthetic benchmark driver
# These kernels will compile but some of them will fail to run
print("Checking next kernel batch...")
synthetic_kernel_batch = glob.glob("../generatedkernels/*.cl")
for synthetic_kernel in synthetic_kernel_batch:
    process = Popen(["../synthetic-benchmark-driver/sbc", synthetic_kernel, "tiny", "0", "0"], stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code == 0: #a usable kernel added it to the list to test for prediction
        usable_kernels.append(synthetic_kernel)
    else:
        unusable_kernels.append(synthetic_kernel)
print("Passed {} and failed {} kernels.".format(len(usable_kernels),len(unusable_kernels)))

# generates AIWC feature spaces for those kernels that run
# also includes libscibench for timing the runtimes of these kernels
successfulkernels = 0
problem_sizes = ["tiny", "small", "medium", "large"]
print("Generating payload AIWC metrics...")
for psize in problem_sizes:
    for kernel in usable_kernels:
        process = Popen(["../../../oclgrind/bin/oclgrind", "--workload-characterisation", "../synthetic-benchmark-driver/sbd", kernel, psize, "0", "0", "runtime"], stdout=PIPE)
        (output, err) = process.communicate()
        exit_code = process.wait()
        print("Finished generating AIWC features")
        
        fdf = getfeatures()
        
        for f in glob.glob("lsb.*"):
            os.remove(f)
        for f in glob.glob('aiwc_*'):
            os.remove(f)
    
        for i in range(0, 50):
            print(" ".join(["../synthetic-benchmark-driver/sbd", kernel, psize, "0", "0", "runtime"]))
            process = Popen(["../synthetic-benchmark-driver/sbd", kernel, psize, "0", "0", "runtime"], stdout=PIPE)
            exit_code = process.wait()
            print("Generating run number:", i)
            
        rdf = getruntimes(psize, "P100")
        
        for f in glob.glob("lsb.*"):
            os.remove(f)
        for f in glob.glob('aiwc_*'):
            os.remove(f)

        jdf = concatandrename(rdf, fdf)
        jdf['device'] = "P100"
        nameoffile = kernel.split("/")[-1].split(".")[0]
        print("saving to", (str(psize) + str(nameoffile) + ".csv"))
        jdf.to_csv(str(psize) + str(nameoffile) + ".csv", index=False)
        
    # read the files just created for this problem size
    # read them into dataframe
    # delete the files
    # rename into format suitable for predictions
    
    
# Concats all the output files from each of our generated kernels together
temp = []
for f in glob.glob("*.csv"):
    if not ("aiwc" in f):
        df = pd.read_csv(f, index_col=None, header=0)
        temp.append((df))
frame = pd.concat(temp, axis=0, ignore_index=True)
frame = orderNames(frame)
frame = frame.dropna()
frame.to_csv("GeneratedKernelFeatures.csv")

Successfully loaded CLgen version 0.1.7
Platform:   Linux
Memory:     95227 MB

Device:     None

Device:     GPU Tesla P100-PCIE-12GB
Compute #.: 56
Frequency:  1328 HZ
Memory:     12198 MB
Driver:     418.87.00

Device:     CPU Intel(R) Xeon(R) Gold 6134 CPU @ 3.20GHz
Compute #.: 32
Frequency:  3200 HZ
Memory:     95227 MB
Driver:     18.1.0.0920
Seed text: __kernel void A(__global float* a, __global float* b, __global float* c, const int d) { 

beginning batch 1 ...


/* === SAMPLE 1 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);
  int g = e + f;
  int h = e + 2 * g;

  b[d * e + f] = (float4)(g, g, g, 1.0f);
}

/* === SAMPLE 2 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  c[e] = a[e] + b[e];
}

/* === SAMPLE 3 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int

__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c) {
    c[e] = a[e] * b[e] * a[e];
  }
}

/* === SAMPLE 28 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e >= d) {
    return;
  }
  c[e] = (e > 0) ? 1.0f + e : 0;

  for (c = 4; c < a; c++) {
    if (a[c] > 0)
      b[e] = a[c] - 1;
    barrier(1);
    if (d == 2) {
      e = a[g];
      k = a[e];
      g = d[e + 1];
      m = a[e + 1];
      a[b] = a[d * 2 - 1];
      a[e + 1] = ao;
      a[a[e]] = a;
      b[r] = b;
      b[c] = b;
    }
    B(a, b, c);
    b[c] = a[b] + 1;
  }
  return 2;
}

/* === SAMPLE 29 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f;
  float g = 0;
  int h = 0;
  int i = 0;
  int j = -1;
  for (int k = 0; k < e; k++) {
    float l = 0;
    float m = 0;
    float n = b

  size_t g = get_global_id(1);

  float h = 0.f;
  int k = 0;

  int l = 0;
  int m;

  for (int l = 0; l < e; l++) {
    const int m = l * f + l;
    for (int n = 0; n < c; n++) {
      float o = 0x1.fffffep127f;
      for (int p = 0; p < 128; p += k) {
        if ((-1) < (p - 1) || ((n + 16) < (c - p))) {
          e[m + q * c + m] = (1.f - p[o][n] * a[n] - n[i * b + q]);
        }
      }
    }
  }
}

/* === SAMPLE 56 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);
  int n = f * 32 + l;

  b[c + f * 16] = a[((d + 1) * b + c) % 1656];
}

/* === SAMPLE 57 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < g) {
    int f = e + c;
    int g = e + g;
    b[d] = f[h] | g;
  }
}

/* === SAMPLE 58 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_

__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c) {
    int f = c + e;
    float g = d * f + e * c;
    c[d] = c[e];
  }
}

/* === SAMPLE 81 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  c[e] = c[e] + g;
}

/* === SAMPLE 82 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);

  if (e < c) {
    b[e] += 21;
  }
}

/* === SAMPLE 83 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);

  if (e < h) {
    a[e] += a[e + 2];
    a[c] = d + 1;
  }
}

/* === SAMPLE 84 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_local_id(0);
  int g = get_local_id(0);

  float h = e.x * (0.222

}

/* === SAMPLE 5 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  if (e >= d) || e >= d) {
    return;
  }

  b[e] = a[e] + b[e];
}

/* === SAMPLE 6 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  int f = get_global_id(1);

  int g = get_global_size(0) * c + e;

  const sampler_t h = 0x0000 | 0x0002 | 0x0000;

  int i = d != 0 ? 1 : 0;
  int j = a[i];
  int k = 0;
  int l = 0;
  int m;

  const int n = 2 * l;

  if ((g >= p || n <= e) || (n >= e - m) {
    int o = l * j + l;
    int p = (p + n) * f + m * r;
    int q = (p - l) * f + j;
    int r = (b[p] + o;
    float s = 0.0;
    for (s = 0; s < l; s++) {
      b[(r + 1) * d + j] = p;
    }
  }
}

/* === SAMPLE 7 === */	
__kernel void A(__global float* a, __global float* b, __global float* c, const int d) {
  int e = get_global_id(0);
  if (e >= d) {
    return;
  }
  c[e] = a[e] 

...

Generating run number: 19
../synthetic-benchmark-driver/sbd ../generatedkernels/18a9a92ea832a38af49acd2531a77ae44ea1f75a.cl small 0 0 runtime
Generating run number: 20
../synthetic-benchmark-driver/sbd ../generatedkernels/18a9a92ea832a38af49acd2531a77ae44ea1f75a.cl small 0 0 runtime
Generating run number: 21
../synthetic-benchmark-driver/sbd ../generatedkernels/18a9a92ea832a38af49acd2531a77ae44ea1f75a.cl small 0 0 runtime
Generating run number: 22
../synthetic-benchmark-driver/sbd ../generatedkernels/18a9a92ea832a38af49acd2531a77ae44ea1f75a.cl small 0 0 runtime
Generating run number: 23
../synthetic-benchmark-driver/sbd ../generatedkernels/18a9a92ea832a38af49acd2531a77ae44ea1f75a.cl small 0 0 runtime
Generating run number: 24
../synthetic-benchmark-driver/sbd ../generatedkernels/18a9a92ea832a38af49acd2531a77ae44ea1f75a.cl small 0 0 runtime
Generating run number: 25
../synthetic-benchmark-driver/sbd ../generatedkernels/18a9a92ea832a38af49acd2531a77ae44ea1f75a.cl small 0 0 runtime
Genera

Generating run number: 27
../synthetic-benchmark-driver/sbd ../generatedkernels/36f41895b8be2fd986c301902e05115f56403941.cl small 0 0 runtime
Generating run number: 28
../synthetic-benchmark-driver/sbd ../generatedkernels/36f41895b8be2fd986c301902e05115f56403941.cl small 0 0 runtime
Generating run number: 29
../synthetic-benchmark-driver/sbd ../generatedkernels/36f41895b8be2fd986c301902e05115f56403941.cl small 0 0 runtime
Generating run number: 30
../synthetic-benchmark-driver/sbd ../generatedkernels/36f41895b8be2fd986c301902e05115f56403941.cl small 0 0 runtime
Generating run number: 31
../synthetic-benchmark-driver/sbd ../generatedkernels/36f41895b8be2fd986c301902e05115f56403941.cl small 0 0 runtime
Generating run number: 32
../synthetic-benchmark-driver/sbd ../generatedkernels/36f41895b8be2fd986c301902e05115f56403941.cl small 0 0 runtime
Generating run number: 33
../synthetic-benchmark-driver/sbd ../generatedkernels/36f41895b8be2fd986c301902e05115f56403941.cl small 0 0 runtime
Genera

Generating run number: 35
../synthetic-benchmark-driver/sbd ../generatedkernels/af60682a29f3468a57f83504c6a32bee200cd5d5.cl small 0 0 runtime
Generating run number: 36
../synthetic-benchmark-driver/sbd ../generatedkernels/af60682a29f3468a57f83504c6a32bee200cd5d5.cl small 0 0 runtime
Generating run number: 37
../synthetic-benchmark-driver/sbd ../generatedkernels/af60682a29f3468a57f83504c6a32bee200cd5d5.cl small 0 0 runtime
Generating run number: 38
../synthetic-benchmark-driver/sbd ../generatedkernels/af60682a29f3468a57f83504c6a32bee200cd5d5.cl small 0 0 runtime
Generating run number: 39
../synthetic-benchmark-driver/sbd ../generatedkernels/af60682a29f3468a57f83504c6a32bee200cd5d5.cl small 0 0 runtime
Generating run number: 40
../synthetic-benchmark-driver/sbd ../generatedkernels/af60682a29f3468a57f83504c6a32bee200cd5d5.cl small 0 0 runtime
Generating run number: 41
../synthetic-benchmark-driver/sbd ../generatedkernels/af60682a29f3468a57f83504c6a32bee200cd5d5.cl small 0 0 runtime
Genera

Generating run number: 43
../synthetic-benchmark-driver/sbd ../generatedkernels/3a89ea1b2c78d7f3fe845b45456cb50eba1f2fe1.cl small 0 0 runtime
Generating run number: 44
../synthetic-benchmark-driver/sbd ../generatedkernels/3a89ea1b2c78d7f3fe845b45456cb50eba1f2fe1.cl small 0 0 runtime
Generating run number: 45
../synthetic-benchmark-driver/sbd ../generatedkernels/3a89ea1b2c78d7f3fe845b45456cb50eba1f2fe1.cl small 0 0 runtime
Generating run number: 46
../synthetic-benchmark-driver/sbd ../generatedkernels/3a89ea1b2c78d7f3fe845b45456cb50eba1f2fe1.cl small 0 0 runtime
Generating run number: 47
../synthetic-benchmark-driver/sbd ../generatedkernels/3a89ea1b2c78d7f3fe845b45456cb50eba1f2fe1.cl small 0 0 runtime
Generating run number: 48
../synthetic-benchmark-driver/sbd ../generatedkernels/3a89ea1b2c78d7f3fe845b45456cb50eba1f2fe1.cl small 0 0 runtime
Generating run number: 49
saving to small3a89ea1b2c78d7f3fe845b45456cb50eba1f2fe1.csv
Finished generating AIWC features
../synthetic-benchmark-drive

Generating run number: 0
../synthetic-benchmark-driver/sbd ../generatedkernels/2de93ad4c08e3c684add565b1fe42013c763e5c5.cl small 0 0 runtime


In [26]:
os.chdir("..")

In [ ]:
%script False

!cp GeneratedKernelFeatures.csv ../predictions
# Moves the generated kernel features over the the predictions folder so that we can carry out openCL predictions with aiwc

In [3]:
!ls
# Gets us back to codes directory

AIWCFeatures			  generatedkernelfeatures
Dockerfile			  generatedkernels
KernelRuntimesP100		  kernels.db
KernelRuntimesXeonGold		  make-kernels-and-features.py
Synthesize-and-run-kernels.ipynb  paper-synthesizing-benchmarks-model
clgen-create-db			  predictions
clgen-fetch-github		  scrape-corpus
corpus				  synthesize_kernels.py
creating_a_corpus.ipynb		  synthetic-benchmark-driver
generatedkernelbatch1		  unzipall


In [7]:
os.chdir("generatedkernelfeatures/")

In [18]:
a = []
counter = 0
for f in glob.glob('*.csv'):
    if (counter == 0):
        a = pd.read_csv(f)
    else:
        a = pd.concat([a, pd.read_csv(f)], axis=0)
    counter+=1

In [84]:
# Parses raw outputs from Oclgrind kernel regions into the names used for opencl predictions with aiwc
#######################################################
kernelDictionary = { 
                    # dwt2d
                    "c_CopySrcToComponents_kernel":"c_CopySrcToComponents", 
                    "kl_fdwt53Kernel_kernel":"cl_fdwt53Kernel", 
                    # csr
                    "csr_kernel":"csr", 
                    # lud
                    "diagonal_kernel":"lud_diagonal", 
                    "perimeter_kernel":"lud_perimeter", 
                    "internel_kernel":"lud_internal", 
                    # srad
                    "srad1_kernel":"srad_cuda_1", 
                    "srad2_kernel":"srad_cuda_2",
                    # kmeans
                    "kmeans_kernel":"kmeansPoint", 
                    "kmeans_kernel":"kmeansPoint", 
                    # openclfft
                    "fft_kernel":"fftRadix16Kernel", 
                    # bfs
                    "kernel1_kernel":"kernel1", 
                    "kernel2_kernel":"kernel2",
                    # crc
                    "kernel_compute_kernel":"crc32_slice8", 
                    # bwa_hmm
                    "_cl_kernel_init_ones_dev_kernel":"init_ones_dev", 
                    "_cl_kernel_init_alpha_dev_kernel":"init_alpha_dev", 
                    "_cl_kernel_init_beta_dev_kernel":"init_beta_dev",
                    "__cl_kernel_scale_alpha_dev_kernel":"scale_alpha_dev",
                    "_cl_kernel_scale_a_dev_kernel":"scale_a_dev", 
                    "_cl_kernel_scale_b_dev_kernel":"scale_b_dev", 
                    "__cl_kernel_s_dot_kernel_naive_kernel":"s_dot_kernel_naive", 
                    "_cl_kernel_calc_alpha_dev_kernel":"calc_alpha_dev", 
                    "_cl_kernel_calc_beta_dev_kernel":"calc_beta_dev", 
                    "_cl_kernel_calc_gamma_dev_kernel":"calc_gamma_dev", 
                    "_cl_kernel_calc_xi_dev_kernel":"calc_xi_dev", 
                    "_cl_kernel_acc_b_dev_kernel":"acc_b_dev",
                    "_cl_kernel_est_a_dev_kernel":"est_a_dev", 
                    "_cl_kernel_est_b_dev_kernel":"est_b_dev", 
                    "_cl_kernel_est_pi_dev_kernel":"est_pi_dev", 
                    "_cl_kernel_sgemvn_kernel_naive_kernel":"mvm_non_kernel_naive", 
                    "_cl_kernel_sgemvt_kernel_naive_kernel":"mvm_trans_kernel_naive", 
                    # needle
                    "clKernel_nw1_kernel":"needle_opencl_shared_1", 
                    "clKernel_nw2_kernel":"needle_opencl_shared_2", 
                    # swat
                    "_cl_kernel_est_pi_dev_kernel":"est_pi_dev", 
                    # gem
                    "gem_kernel":"calc_potential_single_step_dev"  

                   }
    ## missing:
    ## nqueens, swat, kmeans (invert_mapping)

def normaliseRegions(allruntimes):   
    allregionsList = []
    for region in kernelDictionary.keys():
        allruntimes.loc[allruntimes['region'] == region, 'region'] = kernelDictionary[region]
        allregionsList.append(kernelDictionary[region])
    
    return allruntimes[allruntimes['region'].isin(allregionsList)]

In [43]:
#############################
# Loads in the AIWC features
# from the AIWCFeatures directory
#############################

counter = 0
kernals = list()
parsedAlready = set()
for size in ['tiny', 'small', 'medium', 'large', 'default']:
    print("Currently handling ", size)
    for f in glob.glob("AIWCFeatures2309d12/" + size + "/*.csv"):
        kernelString = "_".join(f.split("_")[0:-1])
        if (not (kernelString in parsedAlready)):
            aiwcfeatures = pd.read_csv(f).values.T
            if (counter == 0):
                aiwcfeaturesdf = pd.DataFrame(columns=aiwcfeatures[0,])
                aiwcfeaturesdf['kernel'] = ""
                aiwcfeaturesdf['size'] = ""
            try:
                aiwcfeaturesdf.loc[counter,0:-2] = aiwcfeatures[1,]
                kername = "_".join(f.split('/')[-1].split("_")[1:-1])
                aiwcfeaturesdf.iloc[counter,-2] = kername
                aiwcfeaturesdf.iloc[counter,-1] = size
                counter+=1
            except:
                print(aiwcfeatures[1,])
            parsedAlready.add(kernelString)
            print(kernelString)

aiwcfeaturesdf[aiwcfeaturesdf['size'] == 'medium']
print(aiwcfeaturesdf['size'].unique())
    
aiwcfeaturesdf.groupby(['kernel', 'size']).max().reset_index()

Currently handling  tiny
AIWCFeatures2309d12/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures2309d12/tiny/aiwc_scale_alpha_dev
AIWCFeatures2309d12/tiny/aiwc_calc_xi_dev
AIWCFeatures2309d12/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures2309d12/tiny/aiwc_calc_alpha_dev
AIWCFeatures2309d12/tiny/aiwc_acc_b_dev
AIWCFeatures2309d12/tiny/aiwc_crc32_slice8
AIWCFeatures2309d12/tiny/aiwc_calc_beta_dev
AIWCFeatures2309d12/tiny/aiwc_needle_opencl_shared_1
AIWCFeatures2309d12/tiny/aiwc_calc_gamma_dev
AIWCFeatures2309d12/tiny/aiwc_csr
AIWCFeatures2309d12/tiny/aiwc_lud_internal
AIWCFeatures2309d12/tiny/aiwc_nqueen
AIWCFeatures2309d12/tiny/aiwc_kernel2
AIWCFeatures2309d12/tiny/aiwc_fftRadix16Kernel
AIWCFeatures2309d12/tiny/aiwc_nqueen1
AIWCFeatures2309d12/tiny/aiwc_needle_opencl_shared_2
AIWCFeatures2309d12/tiny/aiwc_kernel1
AIWCFeatures2309d12/tiny/aiwc_init_alpha_dev
AIWCFeatures2309d12/tiny/aiwc_lud_perimeter
AIWCFeatures2309d12/tiny/aiwc_mvm_trans_kernel_naive
AIWCFeatures2309d12/tiny/aiwc_cl_fdwt53Kernel
A

,kernel,size,opcode,total instruction count,freedom to reorder,resource pressure,workitems,operand sum,total # of barriers hit,min instructions to barrier,...,local memory address entropy -- 10 LSBs skipped,total global memory accessed,total local memory accessed,total constant memory accessed,relative local memory usage,relative constant memory usage,total unique branch instructions,90% branch instructions,branch entropy (yokota),branch entropy (average linear)
0,acc_b_dev,tiny,8.0,4.383600e+04,2.66963,1.493360e+01,1024.0,4.383600e+04,0.0,37.0,...,1.676680,240.0,0.0,0.0,0.0000,0.0000,1.0,1.0,32.418200,4.667690e-01
1,c_CopySrcToComponents,large,10.0,7.286077e+08,2.36667,3.000000e+01,9980928.0,7.286077e+08,0.0,73.0,...,16.835700,59885568.0,0.0,0.0,0.0000,0.0000,1.0,1.0,0.000000,0.000000e+00
2,c_CopySrcToComponents,medium,10.0,7.265894e+07,2.36667,3.000000e+01,995328.0,7.265894e+07,0.0,73.0,...,13.509800,5971968.0,0.0,0.0,0.0000,0.0000,1.0,1.0,0.000000,0.000000e+00
3,c_CopySrcToComponents,tiny,10.0,2.900640e+05,2.36422,2.908590e+01,4096.0,2.900640e+05,0.0,30.0,...,5.547940,23328.0,0.0,0.0,0.0000,0.0000,1.0,1.0,5.727520,2.074690e-03
4,calc_alpha_dev,tiny,9.0,7.388000e+03,2.36209,1.104690e+01,256.0,7.388000e+03,0.0,23.0,...,0.918296,180.0,0.0,0.0,0.0000,0.0000,1.0,1.0,5.727520,3.319500e-02
5,calc_beta_dev,tiny,9.0,8.320000e+03,2.33211,1.275000e+01,256.0,8.320000e+03,0.0,25.0,...,1.440810,180.0,0.0,0.0,0.0000,0.0000,1.0,1.0,5.727520,3.319500e-02
6,calc_gamma_dev,tiny,9.0,7.448000e+03,2.33380,1.128120e+01,256.0,7.448000e+03,0.0,23.0,...,1.860960,240.0,0.0,0.0,0.0000,0.0000,1.0,1.0,5.727520,3.319500e-02
7,calc_potential_single_step_dev,tiny,10.0,3.939164e+07,1.95471,6.539500e+02,30720.0,3.939164e+07,0.0,108.0,...,4.027310,2298168.0,0.0,0.0,0.0000,0.0000,10.0,6.0,28.239200,3.822120e-02
8,calc_xi_dev,tiny,10.0,3.695040e+05,2.28180,3.860550e+01,4096.0,3.695040e+05,0.0,39.0,...,4.167410,21600.0,0.0,0.0,0.0000,0.0000,1.0,1.0,28.603200,1.181780e-01
9,cl_fdwt53Kernel,large,8.0,1.009021e+09,2.04756,1.710880e+04,28800.0,1.009021e+09,1411200.0,12.0,...,2.890950,5390982.0,94559982.0,0.0,94.6064,0.0000,31.0,12.0,107.803000,1.771970e-01


In [44]:
print(np.unique(aiwcfeaturesdf['kernel']))

['acc_b_dev' 'c_CopySrcToComponents' 'calc_alpha_dev' 'calc_beta_dev'
 'calc_gamma_dev' 'calc_potential_single_step_dev' 'calc_xi_dev'
 'cl_fdwt53Kernel' 'crc32_slice8' 'csr' 'est_a_dev' 'est_b_dev'
 'est_pi_dev' 'fftRadix16Kernel' 'fftRadix2Kernel' 'fftRadix8Kernel'
 'init_alpha_dev' 'init_beta_dev' 'init_ones_dev' 'invert_mapping'
 'kernel1' 'kernel2' 'kmeansPoint' 'lud_diagonal' 'lud_internal'
 'lud_perimeter' 'mvm_non_kernel_naive' 'mvm_trans_kernel_naive'
 'needle_opencl_shared_1' 'needle_opencl_shared_2' 'nqueen' 'nqueen1'
 's_dot_kernel_naive' 'scale_a_dev' 'scale_alpha_dev' 'scale_b_dev'
 'srad_cuda_1' 'srad_cuda_2']


In [48]:
#############################
# Loads in the AIWC features
# from the AIWCFeatures directory
# includes invocation numbers 
#############################

counter = 0
kernals = list()
parsedAlready = set()
currentdf = []
for size in ['tiny', 'small', 'medium', 'large', 'default']:
    print("Currently handling ", size)
    for f in glob.glob("AIWCFeatures/" + size + "/*.csv"):
        kernelString = "_".join(f.split("_")[0:-1])
        print(kernelString)
        invonum = int(re.findall(r'\d+', f)[-1])

        featuresdf = pd.read_csv(f).values.T
        fdf = pd.DataFrame(columns=featuresdf[0,])
        fdf.loc[0] = featuresdf[1,]
        fdf['invocation'] = invonum
        fdf['kernel'] = "_".join(f.split('/')[-1].split("_")[1:-1])
        fdf['size'] = size
        if (len(currentdf) == 0):
            currentdf = fdf
        else:
            currentdf = pd.concat([currentdf, fdf], axis=0)

Currently handling  tiny
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_crc32_slice8
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_crc32_slice8
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_needle_opencl_shared_1
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_crc32_slice8
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_s

AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_crc32_slice8
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_gamma_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_fftRadix16Kernel
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIW

AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_calc_gamma_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_csr
AIWCFeatures/tiny/aiwc_s_dot

AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_crc32_slice8
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_gamma_dev
AIWCFeatures/tiny/a

AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_kernel2
AIWCFeatures/tiny/aiwc_calc_gamma_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_kernel2
AIWCFeatures/tiny/aiwc_kernel1
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFea

...

AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kernel1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_cl_fdwt53Kernel
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large

AIWCFeatures/large/aiwc_csr
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_memory_transfers
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_open

AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large

AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kernel1
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_cl_fdwt53Kernel
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_sh

AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_kmeansPoint
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
AIWCFeatures/large/aiwc_needle_opencl_shared_2
AIWCFeatures/large/aiwc_needle_opencl_shared_1
Currently handling  default


In [17]:
#############################
# Loads in the Opendwarfs Kernel runtime features into allruntimes
#############################

start = True
allruntimes = []
parsed = []
for root, dirs, files in os.walk("KernelRuntimesXeonGold/"):
#     print(root, dirs)
    # Iterates through a single instance of a file and adds all of the information to a dataframe
    for file in files:
        file = os.path.join(root,file)
        try:
            testfolder = pd.read_csv(file, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)

            totalTimes = testfolder.groupby("repeats_to_two_seconds").sum().reset_index()
            totalTimes = totalTimes.rename(columns={"time":"total_time"})
            totalTimes = totalTimes[['total_time','repeats_to_two_seconds']]
            testfolder = pd.merge(testfolder, totalTimes, on='repeats_to_two_seconds')
            testfolder = testfolder.rename(columns={"time":"kernel_time"})

            problemSize = ""
            if ('tiny' in file):
                problemSize = 'tiny'
            elif ('small' in file):
                problemSize = 'small'
            elif ('medium' in file):
                problemSize = 'medium'
            elif ('large' in file):
                problemSize = 'large'

            testfolder['size'] = problemSize

            testfolder = testfolder.groupby(['region', 'size']).mean().reset_index()

            testfolder['device'] = file.split("/")[1].split("_")[0]
            testfolder['application'] = file.split("/")[1].split("_")[1]
            testfolder['kernel'] = file.split("/")[-1].split('.')[1]    

            testfolder = testfolder[['application','region','total_time','kernel_time','size','kernel','device']]

            print(file, testfolder.values.shape)
            if (start):
                allruntimes = testfolder
                start = False
                dickhead += 1
            else:
                allruntimes = pd.concat([allruntimes, testfolder], axis=0)
        except:
            print("Failed on:", file)


KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-24 (8, 7)
Failed on: KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-24
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-33 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-19 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-16 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-25 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-43 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-34 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-15 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-13 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-46 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-36 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-18 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-10 (8, 7)
Ke

KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-23 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-28 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-41 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-33 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-9 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-1 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-5 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-48 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-15 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-16 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-7 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-38 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-20 (9, 7)
Ker

KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-38 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-20 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-47 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-3 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-4 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-30 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-37 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-32 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-31 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-2 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-34 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-12 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-19 (9, 7)
Ke

KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-34 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-24 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-3 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-25 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-35 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-12 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-49 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-31 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-9 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-39 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-21 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-15 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-14 (7, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-32 (7, 7)
KernelRu

KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-28 (39, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-41 (39, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-23 (39, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-1 (39, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-2 (39, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-4 (39, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-26 (39, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_medium_time.0/lsb.kmeans.r0-7 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_medium_time.0/lsb.kmeans.r0-43 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_medium_time.0/lsb.kmeans.r0-42 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_medium_time.0/lsb.kmeans.r0-32 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_medium_time.0/lsb.kmeans.r0-3 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_medium_time.0/l

...

KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-28 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-36 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-41 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-21 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-9 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-40 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-26 (7, 7)
Failed on: KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-1
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-13 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-49 (7, 7)
Failed on: KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-5
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-25 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_tiny_time.0/lsb.kmeans.r0-12 (7, 7)
KernelR

KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-28 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-43 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-42 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-27 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-9 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-12 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-32 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-34 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-35 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-14 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-25 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_tiny_time.0/lsb.openclfft.r0-23 (5, 7)
KernelRuntimesXeo

KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-6 (13, 7)
KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-49 (13, 7)
KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-31 (13, 7)
KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-22 (13, 7)
KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-36 (13, 7)
KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-17 (13, 7)
KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-40 (13, 7)
KernelRuntimesXeonGold/gold-6134_lud_medium_time.0/lsb.lud.r0-15 (13, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-2 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-33 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-3 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-7 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0 (5, 7)
KernelRu

KernelRuntimesXeonGold/gold-6134_needle_tiny_time.0/lsb.needle.r0-14 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_tiny_time.0/lsb.needle.r0-17 (9, 7)
KernelRuntimesXeonGold/gold-6134_needle_tiny_time.0/lsb.needle.r0-6 (9, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-39 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-22 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-40 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-25 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-2 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-21 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-17 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-14 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-48 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-12 (7, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-46 (7, 7)
Kern

KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-29 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-16 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-44 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-28 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-36 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-41 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-21 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-9 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-40 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-26 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-1 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-13 (7, 7)
KernelRuntimesXeonGold/gold-6134_kmeans_large_time.0/lsb.kmeans.r0-49 (7, 7)
K

In [18]:
normalisedAllRuntimes = normaliseRegions(allruntimes)
normalisedAllRuntimes = normalisedAllRuntimes.rename(columns={"region":"kernel"})
daaaata = pd.merge(aiwcfeaturesdf, normalisedAllRuntimes, how='inner', on=['kernel', 'size'])
daaaata['run'] = np.arange(1, daaaata.values.shape[0]+1, 1)
daaaata = renamecolumns(daaaata)

In [20]:
daaaata.to_csv("predictions/ictions/XeonGoldData.csv", index=False)

In [16]:
daaaata.to_csv("predictions/P100Data.csv", index=False)

In [11]:
daaaata

,application,size,opcode,granularity,barriers_per_instruction,instructions_per_operand,total_instruction_count,workitems,operand_sum,total_barriers_hit,...,total_unique_branch_instructions,ninety_percent_branch_instructions,branch_entropy_yokota,branch_entropy_average_linear,device,total_time,kernel_time,invocation,kernel,run
0,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,392040.076191,20.737593,0,s_dot_kernel_naive,1
1,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,387217.234330,20.404137,0,s_dot_kernel_naive,2
2,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,383740.163971,20.180443,0,s_dot_kernel_naive,3
3,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,374068.347567,19.553564,0,s_dot_kernel_naive,4
4,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,392837.339732,20.759111,0,s_dot_kernel_naive,5
5,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,374727.597566,19.508622,0,s_dot_kernel_naive,6
6,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,391707.511810,20.623014,0,s_dot_kernel_naive,7
7,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,392213.253764,20.709774,0,s_dot_kernel_naive,8
8,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,383063.046868,20.070074,0,s_dot_kernel_naive,9
9,bwa,small,7,9.21319e-06,9.21319e-06,1,108540,7680,108540,0,...,1,1,9.29151,0.015534,p100,379992.585948,19.822159,0,s_dot_kernel_naive,10


In [7]:
testfolder

,application,region,total_time,kernel_time,size,kernel
0,openclfft,device_side_buffer_setup,201.895451,1.657294,tiny,openclfft
1,openclfft,device_side_d2h_copy,201.895451,20.029226,tiny,openclfft
2,openclfft,device_side_h2d_copy,201.895451,133.059411,tiny,openclfft
3,openclfft,fft_kernel,201.895451,46.807445,tiny,openclfft
4,openclfft,host_side_setup,2843.884766,2402.392578,tiny,openclfft


In [71]:
#############################
# Loads in the AIWC features
# from the AIWCFeatures directory
#############################

counter = 0
kernals = list()
parsedAlready = set()
for size in ['tiny', 'small', 'medium', 'large', 'default']:
    print("Currently handling ", size)
    for f in glob.glob("AIWCFeatures/" + size + "/*.csv"):
        kernelString = "_".join(f.split("_")[0:-1])
        if (not (kernelString in parsedAlready)):
            aiwcfeatures = pd.read_csv(f).values.T
            if (counter == 0):
                aiwcfeaturesdf = pd.DataFrame(columns=aiwcfeatures[0,])
                aiwcfeaturesdf['kernel'] = ""
                aiwcfeaturesdf['size'] = ""
            try:
                aiwcfeaturesdf.loc[counter,0:-2] = aiwcfeatures[1,]
                kername = "_".join(f.split('/')[-1].split("_")[1:-1])
                aiwcfeaturesdf.iloc[counter,-2] = kername
                aiwcfeaturesdf.iloc[counter,-1] = size
                counter+=1
            except:
                print(aiwcfeatures[1,])
            parsedAlready.add(kernelString)
            print(kernelString)

aiwcfeaturesdf[aiwcfeaturesdf['size'] == 'medium']
print(aiwcfeaturesdf['size'].unique())
    
aiwcfeaturesdf.groupby(['kernel', 'size']).max().reset_index()

Currently handling  tiny
AIWCFeatures/tiny/aiwc_s_dot_kernel_naive
AIWCFeatures/tiny/aiwc_crc32_slice8
AIWCFeatures/tiny/aiwc_calc_xi_dev
AIWCFeatures/tiny/aiwc_mvm_non_kernel_naive
AIWCFeatures/tiny/aiwc_calc_alpha_dev
AIWCFeatures/tiny/aiwc_acc_b_dev
AIWCFeatures/tiny/aiwc_calc_beta_dev
AIWCFeatures/tiny/aiwc_needle_opencl_shared_1
AIWCFeatures/tiny/aiwc_calc_gamma_dev
AIWCFeatures/tiny/aiwc_csr
AIWCFeatures/tiny/aiwc_scale_alpha_dev
AIWCFeatures/tiny/aiwc_lud_internal
AIWCFeatures/tiny/aiwc_nqueen
AIWCFeatures/tiny/aiwc_kernel2
AIWCFeatures/tiny/aiwc_fftRadix16Kernel
AIWCFeatures/tiny/aiwc_nqueen1
AIWCFeatures/tiny/aiwc_needle_opencl_shared_2
AIWCFeatures/tiny/aiwc_kernel1
AIWCFeatures/tiny/aiwc_init_alpha_dev
AIWCFeatures/tiny/aiwc_lud_perimeter
AIWCFeatures/tiny/aiwc_mvm_trans_kernel_naive
AIWCFeatures/tiny/aiwc_cl_fdwt53Kernel
AIWCFeatures/tiny/aiwc_srad_cuda_2
AIWCFeatures/tiny/aiwc_scale_a_dev
AIWCFeatures/tiny/aiwc_kmeansPoint
AIWCFeatures/tiny/aiwc_est_pi_dev
AIWCFeatures/tin

,kernel,size,opcode,total instruction count,freedom to reorder,resource pressure,workitems,operand sum,total # of barriers hit,min instructions to barrier,...,local memory address entropy -- 10 LSBs skipped,total global memory accessed,total shared memory accessed,total private memory accessed,relative shared memory usage,relative private memory usage,total unique branch instructions,90% branch instructions,branch entropy (yokota),branch entropy (average linear)
0,MatchStringGPUSync,small,10.0,9.312309e+07,4.92128,2.956640e+05,64.0,9.312310e+07,262016.0,4.0,...,10.739100,14242010.0,0.0,14242010.0,0.0000,100.000,12.0,7.0,127031.00000,4.467780e-01
1,acc_b_dev,small,7.0,1.110012e+06,8.50000,4.111840e-03,58368.0,1.110010e+06,0.0,19.0,...,1.641630,180.0,0.0,180.0,0.0000,100.000,1.0,1.0,12.95500,9.596760e-04
2,acc_b_dev,tiny,7.0,2.047600e+04,8.50000,2.343750e-01,1024.0,2.047600e+04,0.0,19.0,...,1.680480,180.0,0.0,180.0,0.0000,100.000,1.0,1.0,12.95500,5.550050e-02
3,c_CopySrcToComponents,large,10.0,3.593134e+08,5.66667,6.000000e+00,9980928.0,3.593130e+08,0.0,36.0,...,16.842700,59885568.0,0.0,59885568.0,0.0000,100.000,1.0,1.0,0.00000,0.000000e+00
4,c_CopySrcToComponents,medium,10.0,3.583181e+07,5.66667,6.000000e+00,995328.0,3.583180e+07,0.0,36.0,...,13.507900,5971968.0,0.0,5971968.0,0.0000,100.000,1.0,1.0,0.00000,0.000000e+00
5,c_CopySrcToComponents,small,10.0,1.082080e+06,5.66667,5.958690e+00,30208.0,1.082080e+06,0.0,10.0,...,8.459730,180000.0,0.0,180000.0,0.0000,100.000,1.0,1.0,11.45500,5.299240e-04
6,c_CopySrcToComponents,tiny,10.0,1.420480e+05,5.66667,5.695310e+00,4096.0,1.420480e+05,0.0,10.0,...,5.562000,23328.0,0.0,23328.0,0.0000,100.000,1.0,1.0,11.45500,3.920610e-03
7,calc_alpha_dev,small,9.0,3.024000e+03,6.33333,7.031250e-01,256.0,3.024000e+03,0.0,9.0,...,0.918296,120.0,0.0,120.0,0.0000,100.000,1.0,1.0,5.72752,3.319500e-02
8,calc_alpha_dev,tiny,9.0,3.024000e+03,6.33333,7.031250e-01,256.0,3.024000e+03,0.0,9.0,...,0.918296,120.0,0.0,120.0,0.0000,100.000,1.0,1.0,5.72752,3.319500e-02
9,calc_beta_dev,small,9.0,3.444000e+03,8.66667,7.031250e-01,256.0,3.444000e+03,0.0,9.0,...,0.918296,180.0,0.0,180.0,0.0000,100.000,1.0,1.0,5.72752,3.319500e-02


In [73]:
aiwcfeaturesdf = renamecolumns(aiwcfeaturesdf)

In [75]:
a = pd.read_csv("predictions/recompiledAIWCData.csv")
newDat = a[['kernel_time','total_time','device','kernel','application','size']]

In [76]:
recompiledData = pd.merge(aiwcfeaturesdf, newDat, how='inner', on=['size','kernel'])

In [78]:
recompiledData.to_csv("predictions/recompiledAIWCData.csv", index=False)

In [95]:
# file = "KernelRuntimesP100/p100_kmeans_medium_time.0/lsb.kmeans.r0-7"
file = "KernelRuntimesP100/p100_lud_medium_time.0/lsb.lud.r0-8"

testfolder = pd.read_csv(file, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)

totalTimes = testfolder.groupby("repeats_to_two_seconds").sum().reset_index()
totalTimes = totalTimes.rename(columns={"time":"total_time"})
totalTimes = totalTimes[['total_time','repeats_to_two_seconds']]
testfolder = pd.merge(testfolder, totalTimes, on='repeats_to_two_seconds')
testfolder = testfolder.rename(columns={"time":"kernel_time"})

problemSize = ""
if ('tiny' in file):
    problemSize = 'tiny'
elif ('small' in file):
    problemSize = 'small'
elif ('medium' in file):
    problemSize = 'medium'
elif ('large' in file):
    problemSize = 'large'

testfolder['size'] = problemSize
testfolder.columns

Index(['matrix_dimension', 'repeats_to_two_seconds', 'region', 'id',
       'kernel_time', 'overhead', 'total_time', 'size'],
      dtype='object')

In [115]:
def extractInvocation(lsbFile):
    kernelsSeen = dict()
    invocation = []
    currentRepeat = 0
    for index, row in lsbFile.iterrows():
        if (currentRepeat != row['repeats_to_two_seconds']):
            kernelsSeen = dict()
            currentRepeat = row['repeats_to_two_seconds']
        reg = row['region']
        if (reg in kernelsSeen):
            kernelsSeen[reg] += 1
        else:
            kernelsSeen[reg] = 0
        invocation.append(kernelsSeen[reg])
    return invocation

invo = extractInvocation(testfolder)

In [93]:
filesss = []
for f in glob.glob("AIWCFeatures/medium/*.csv"):
#     print(f)
    if ('lud_internal' in f):
        filesss.append(f)
filesss.sort()

In [ ]:
#############################
# Loads in the Opendwarfs Kernel runtime features into allruntimes
# handles this by invocation
#############################

def extractInvocation(lsbFile):
    kernelsSeen = dict()
    invocation = []
    currentRepeat = 0
    for index, row in lsbFile.iterrows():
        if (currentRepeat != row['repeats_to_two_seconds']):
            kernelsSeen = dict()
            currentRepeat = row['repeats_to_two_seconds']
        reg = row['region']
        if (reg in kernelsSeen):
            kernelsSeen[reg] += 1
        else:
            kernelsSeen[reg] = 0
        invocation.append(kernelsSeen[reg])
    return invocation

start = True
allruntimes = []
parsed = []
for root, dirs, files in os.walk("KernelRuntimesXeonGold/"):
    for file in files:
        file = os.path.join(root,file)
        try:
            testfolder = pd.read_csv(file, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
            invo = extractInvocation(testfolder)

            totalTimes = testfolder.groupby("repeats_to_two_seconds").sum().reset_index()
            totalTimes = totalTimes.rename(columns={"time":"total_time"})
            totalTimes = totalTimes[['total_time','repeats_to_two_seconds']]
            testfolder = pd.merge(testfolder, totalTimes, on='repeats_to_two_seconds')
            testfolder = testfolder.rename(columns={"time":"kernel_time"})

            problemSize = ""
            if ('tiny' in file):
                problemSize = 'tiny'
            elif ('small' in file):
                problemSize = 'small'
            elif ('medium' in file):
                problemSize = 'medium'
            elif ('large' in file):
                problemSize = 'large'

            testfolder['size'] = problemSize
            testfolder['device'] = file.split("/")[1].split("_")[0]
            testfolder['application'] = file.split("/")[1].split("_")[1]
            testfolder['kernel'] = file.split("/")[-1].split('.')[1]   

            testfolder['invocation'] = invo

            testfolder = testfolder[['application','region','total_time','kernel_time','size','kernel','device','invocation']]
            testfolder = testfolder.groupby(['application','invocation','size','device','region']).mean().reset_index()

            print(file, testfolder.values.shape)
            if (start):
                allruntimes = testfolder
                start = False
                dickhead += 1
            else:
                allruntimes = pd.concat([allruntimes, testfolder], axis=0)
        except:
            print("Failed on:", file)


KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-24 (55, 7)
Failed on: KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-24
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-33 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-19 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-16 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-25 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-43 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-34 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-15 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-13 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-46 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-36 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-18 (55, 7)
KernelRuntimesXeonGold/gold-6134_bfs_large_time.0/lsb.bfs.r0-

KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-23 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-28 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-41 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-33 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-9 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-1 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-5 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-48 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-15 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-16 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-7 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_large_time.0/lsb.needle.r0-38 (1027, 7)
KernelRuntimesXeonGold/gold-6134_needle_larg

KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-48 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-15 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-16 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-7 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-38 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-20 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-47 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-3 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-4 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-30 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-37 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-32 (47, 7)
KernelRuntimesXeonGold/gold-6134_needle_small_time.0/lsb.needle.r0-

KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-26 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-5 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-38 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-28 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-44 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-16 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-30 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-33 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-34 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-24 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-3 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-25 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-35 (9, 7)
KernelRuntimesXeonGold/gold-6134_csr_large_time.0/lsb.csr.r0-12 (9, 7)
KernelRu

KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-22 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-48 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-3 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-40 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-6 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-25 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-46 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-8 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-28 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-41 (31020, 7)
KernelRuntimesXeonGold/gold-6134_bwa_hmm_small_time.0/lsb.bwa_hmm.r0-23 (31020, 7)
KernelRunt

...

KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-14 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-32 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-4 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-15 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-29 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-31 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-27 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-39 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-40 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-44 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-12 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-34 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-13 (6, 7)
KernelRuntimesXeonGold/gold-6134_crc_medium_time.0/lsb.crc.r0-47 

KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-44 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-13 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-32 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-42 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-1 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-37 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-26 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-16 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-34 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-47 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-18 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.gem.r0-45 (16, 7)
KernelRuntimesXeonGold/gold-6134_gem_medium_time.0/lsb.

KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-34 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-47 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-18 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-45 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-43 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-35 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-31 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-6 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-24 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-23 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-49 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-19 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_small_time.0/lsb.gem.r0-41 (8, 7)
KernelRunt

KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-12 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-32 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-34 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-35 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-14 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-25 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-23 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-29 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-8 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-26 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-38 (5, 7)
KernelRuntimesXeonGold/gold-6134_openclfft_small_time.0/lsb.openclfft.r0-15 (5, 7)
Kerne

KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-36 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-7 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-3 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-27 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-33 (8, 7)
KernelRuntimesXeonGold/gold-6134_gem_tiny_time.0/lsb.gem.r0-11 (8, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-24 (51, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-33 (51, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-19 (51, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-16 (51, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-25 (51, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-43 (51, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-34 (51, 7)
KernelRuntimesXeonGold/gold-6134_bfs_medium_time.0/lsb.bfs.r0-15 (51, 7

In [83]:
currentdf = pd.read_csv("Temporarydataanalysisfiles/aiwcfeatures-invocation.csv")
# currentdf = currentdf.drop(["transfer: device to host","transfer: host to device"], axis=1)
# currentdf = currentdf.dropna()
# currentdf.to_csv("Temporarydataanalysisfiles/aiwcfeatures-invocation.csv", index=False)

In [175]:
kernelruntimes = normaliseRegions(allruntimes)

In [86]:
kernelruntimes.to_csv("Temporarydataanalysisfiles/gold6134-runtimes-invocation", index=False)

In [176]:
kernelruntimes = kernelruntimes.rename(columns={'region':'kernel'})

In [177]:
kernelruntimes = kernelruntimes.groupby(['kernel','invocation','size','application','device']).mean().reset_index()

In [178]:
mergedStuff = pd.merge(kernelruntimes, currentdf, how='inner', left_on=['invocation','size','kernel'],right_on=['invocation','size','kernel'])

In [179]:
mergedStuff = renamecolumns(mergedStuff)
mergedStuff = orderNames(mergedStuff)

In [180]:
mergedStuff.columns

Index(['application', 'size', 'opcode', 'granularity',
       'barriers_per_instruction', 'instructions_per_operand',
       'total_instruction_count', 'workitems', 'operand_sum',
       'total_barriers_hit', 'min_instructions_to_barrier',
       'max_instructions_to_barrier', 'median_instructions_to_barrier',
       'max_simd_width', 'mean_simd_width', 'stddev_simd_width',
       'total_memory_footprint', 'ninety_percent_memory_footprint',
       'global_memory_address_entropy', 'local_memory_address_entropy_1',
       'local_memory_address_entropy_2', 'local_memory_address_entropy_3',
       'local_memory_address_entropy_4', 'local_memory_address_entropy_5',
       'local_memory_address_entropy_6', 'local_memory_address_entropy_7',
       'local_memory_address_entropy_8', 'local_memory_address_entropy_9',
       'local_memory_address_entropy_10', 'total_unique_branch_instructions',
       'ninety_percent_branch_instructions', 'branch_entropy_yokota',
       'branch_entropy_average_li

In [133]:
mergedStuff.to_csv("predictions/gold6134Data-invocations.csv",index=False)

In [92]:
def extractInvocation(lsbFile):
    kernelsSeen = dict()
    invocation = []
    currentRepeat = 0
    for index, row in lsbFile.iterrows():
        if (currentRepeat != row['repeats_to_two_seconds']):
            kernelsSeen = dict()
            currentRepeat = row['repeats_to_two_seconds']  
        reg = str(row['region'])
        if (reg in kernelsSeen.keys()):
            kernelsSeen[reg] += 1
        else:
            kernelsSeen[reg] = 0
        invocation.append(kernelsSeen[reg])
    return invocation

file = "KernelRuntimesP100/p100_bfs_large_time.0/lsb.bfs.r0-24"
testfolder = pd.read_csv(file, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
invo = extractInvocation(testfolder)

totalTimes = testfolder.groupby("repeats_to_two_seconds").sum().reset_index()
totalTimes = totalTimes.rename(columns={"time":"total_time"})
totalTimes = totalTimes[['total_time','repeats_to_two_seconds']]
testfolder = pd.merge(testfolder, totalTimes, on='repeats_to_two_seconds')
testfolder = testfolder.rename(columns={"time":"kernel_time"})

problemSize = ""
if ('tiny' in file):
    problemSize = 'tiny'
elif ('small' in file):
    problemSize = 'small'
elif ('medium' in file):
    problemSize = 'medium'
elif ('large' in file):
    problemSize = 'large'

testfolder['size'] = problemSize
testfolder['device'] = file.split("/")[1].split("_")[0]
testfolder['application'] = file.split("/")[1].split("_")[1]
testfolder['kernel'] = file.split("/")[-1].split('.')[1]   

testfolder['invocation'] = invo

testfolder = testfolder[['application','region','total_time','kernel_time','size','kernel','device','invocation']]
testfolder = testfolder.groupby(['application','invocation','size','device','region']).mean().reset_index()


In [194]:
file = "oldlsbfiles/titanx_openclfft_medium_time.0/lsb.openclfft.r0-2"
a = pd.read_csv(file, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
a

,signal_length,region,id,time,overhead
0,524288,host_side_setup,0,43258.496094,0
1,524288,device_side_buffer_setup,0,7.308594,0
2,524288,device_side_h2d_copy,0,1372.578125,0
3,524288,fft_kernel,0,645.974609,0
4,524288,device_side_d2h_copy,0,743.355469,0


In [211]:
import copy
b = pd.read_csv("predictions/full_old_data.csv")
print(b.columns)

Index(['Unnamed: 0', 'application', 'size', 'kernel', 'opcode', 'granularity',
       'barriers_per_instruction', 'instructions_per_operand',
       'total_instruction_count', 'workitems', 'operand_sum',
       'total_barriers_hit', 'min_instructions_to_barrier',
       'max_instructions_to_barrier', 'median_instructions_to_barrier',
       'max_simd_width', 'mean_simd_width', 'stddev_simd_width',
       'total_memory_footprint', 'ninety_percent_memory_footprint',
       'global_memory_address_entropy', 'local_memory_address_entropy_1',
       'local_memory_address_entropy_2', 'local_memory_address_entropy_3',
       'local_memory_address_entropy_4', 'local_memory_address_entropy_5',
       'local_memory_address_entropy_6', 'local_memory_address_entropy_7',
       'local_memory_address_entropy_8', 'local_memory_address_entropy_9',
       'local_memory_address_entropy_10', 'total_unique_branch_instructions',
       'ninety_percent_branch_instructions', 'branch_entropy_yokota',
       'b

In [212]:
c = copy.deepcopy(b[['kernel_time','total_time','device','kernel','size','application']])

In [217]:
d = pd.merge(c, aiwcfeaturesdf, how='inner',left_on=['kernel','size'], right_on=['kernel','size'])

In [223]:
d = renamecolumns(d)
d['invocation'] = 0 
d = orderNames(d)

In [225]:
d.to_csv("predictions/recompiledAIWCData.csv", index=False)

In [37]:
stuffWithRepeats = set()
stuffWithoutRepeats = set()

for root, dirs, files in os.walk("oldlsbfiles/"):
    for file in files:
        if ('lsb' in file):
            file = os.path.join(root,file)
            name = file.split("/")[3].split(".")[1]
            print('evaluating:',name)
            if (not ((name in stuffWithRepeats) or (name in stuffWithoutRepeats))):
                a = pd.read_csv(file, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
                if ("repeats_to_two_seconds" in a.columns):
                    stuffWithRepeats.add(name)
                    print(name, " has repeats")
                else:
                    stuffWithoutRepeats.add(name)
                    print(name, " absolutely totally does not have repeats")


evaluating: gem
gem  absolutely totally does not have repeats
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: kmeans
kmeans  absolutely totally does not have repeats
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating

needle  has repeats
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: needle
evaluating: dwt2d
evaluating: dwt2d
evaluating: d

evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: lud
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: kmeans
evaluating: k

bfs  has repeats
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: bfs
evaluating: openclfft
evaluating: openclfft
evaluating: openclfft
evaluating: openclfft
evaluating: openclfft
evaluating: openclfft
evaluating: openclfft
evaluating: openclfft
evaluating: openclfft
e

evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: swat
evaluating: sw

...

evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: srad
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem


evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
evaluating: dwt2d
eval

evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluati

evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluating: csr
evaluati

evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: crc
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluating: gem
evaluati

In [18]:
f = "oldlsbfiles/time_data/xeon_es-2697v2_swat_small_time.0/lsb.swat.r0-34"
a = pd.read_csv(f, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
print("repeats_to_two_seconds" in a.columns)

True


In [17]:
f = "oldlsbfiles/time_data/p100_swat_small_time.0/lsb.swat.r0-34"
b = pd.read_csv(f, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
b

,region,repeats_to_two_seconds,id,time,overhead
0,kernel_creation,NaN,0,3886.812988,0
1,device_side_buffer_setup,NaN,0,8.222412,0
2,host_side_setup,NaN,0,92.156250,0
3,device_side_buffer_setup,NaN,0,5.133057,0
4,host_side_setup,NaN,0,10.635254,0
5,device_side_buffer_setup,NaN,0,7.935791,0
6,host_side_setup,NaN,0,46.743408,0
7,device_side_buffer_setup,NaN,0,16.434814,0
8,host_side_setup,NaN,0,0.185059,0
9,device_side_buffer_setup,NaN,0,7.165283,0


In [82]:
def extractInvocation(lsbFile):
    kernelsSeen = dict()
    invocation = []
    currentRepeat = 0
    for index, row in lsbFile.iterrows():
        if ("repeats_to_two_seconds" in lsbFile.columns):
            if (currentRepeat != row['repeats_to_two_seconds']):
                kernelsSeen = dict()
                currentRepeat = row['repeats_to_two_seconds']
        reg = row['region']
        if (reg in kernelsSeen):
            kernelsSeen[reg] += 1
        else:
            kernelsSeen[reg] = 0
        invocation.append(kernelsSeen[reg])
    return invocation


file = "oldlsbfiles/time_data/gtx1080_kmeans_tiny_time.0/lsb.kmeans.r0-7"
testfolder = pd.read_csv(file, skiprows=9, sep="[ ]+", engine='python', skipfooter=1)
invo = extractInvocation(testfolder)
try:
    totalTimes = testfolder.groupby("repeats_to_two_seconds").sum().reset_index()
    totalTimes = totalTimes.rename(columns={"time":"total_time"})

    totalTimes = totalTimes[['total_time','repeats_to_two_seconds']]
    testfolder = pd.merge(testfolder, totalTimes, on='repeats_to_two_seconds')
except:
    testfolder["total_time"] = np.sum(testfolder['time'])
    print("Repeats_to_to_seconds not detected, numerically likely unstable in: ", file)
testfolder = testfolder.rename(columns={"time":"kernel_time"})

problemSize = ""
if ('tiny' in file):
    problemSize = 'tiny'
elif ('small' in file):
    problemSize = 'small'
elif ('medium' in file):
    problemSize = 'medium'
elif ('large' in file):
    problemSize = 'large'

testfolder['size'] = problemSize
testfolder['device'] = file.split("/")[1].split("_")[0]
testfolder['application'] = file.split("/")[1].split("_")[1]
testfolder['kernel'] = file.split("/")[-1].split('.')[1]   

testfolder['invocation'] = invo

testfolder = testfolder[['application','region','total_time','kernel_time','size','kernel','device','invocation']]
testfolder = testfolder.groupby(['application','invocation','size','device','region']).mean().reset_index()

Repeats_to_to_seconds not detected, numerically likely unstable in:  oldlsbfiles/time_data/gtx1080_kmeans_tiny_time.0/lsb.kmeans.r0-7


In [85]:
normaliseRegions(testfolder)

,application,invocation,size,device,region,total_time,kernel_time
6,data,0,tiny,time,kmeansPoint,122962.615726,25.197266
9,data,1,tiny,time,kmeansPoint,122962.615726,20.563965
12,data,2,tiny,time,kmeansPoint,122962.615726,19.023438
15,data,3,tiny,time,kmeansPoint,122962.615726,19.126465
18,data,4,tiny,time,kmeansPoint,122962.615726,18.779785
21,data,5,tiny,time,kmeansPoint,122962.615726,19.020020
24,data,6,tiny,time,kmeansPoint,122962.615726,18.905762
27,data,7,tiny,time,kmeansPoint,122962.615726,19.060547
30,data,8,tiny,time,kmeansPoint,122962.615726,18.820801
33,data,9,tiny,time,kmeansPoint,122962.615726,18.523438
